In [1]:
import numpy as np

from bokeh.models import ColumnDataSource, DataRange1d, Plot, LinearAxis, Grid
from bokeh.models.glyphs import AnnularWedge, Ellipse
from bokeh.io import curdoc, show
import queue

In [ ]:
def sunburstf(nodes, total=np.pi * 2, level = 0, ax=None):
    ax = ax
    width = 0.5
    
    for n in nodes:
        n.rang = #relative part of range parent according to leaves subtree
        glyph = AnnularWedge(x='x', y='y', inner_radius=(level * width), outer_radius=(inner_radius+width),
                             start_angle=right_bound, end_angle=(right_bound+rang))

In [ ]:
p=0
for i in range(1,4):
    p = p+i
p

In [ ]:
xdr = DataRange1d()
ydr = DataRange1d()
data = {'x': [1,2,3,4], 'y': [1,2,3,4]}
source = ColumnDataSource(data)
plot = Plot(
    title=None, x_range=xdr, y_range=ydr, plot_width=300, plot_height=300,
    h_symmetry=False, v_symmetry=False, min_border=0, toolbar_location=None)
glyph = AnnularWedge(x='x', y='y', inner_radius=0.2, outer_radius=3, start_angle=0, end_angle=4)
plot.add_glyph(source, glyph)
curdoc().add_root(plot)
show(test_plot)

In [ ]:
sun_que = queue.Queue()
b = [4,5,7,9]
sun_que.put(b)
if len(sun_que.get()) == 4:
    sun_que.get(timeout=10)

In [ ]:
#create a queue of all the nodes sorted on level
#use this queue for BFS to create visualization
for i in levels:
    for n in nodes:
        if level[n] == i:
            que.put(n)

In [ ]:
#add a glyph for every node to the sunburst
n_old
while not que.empty():
    n = que.get()
    glyph = AnnularWedge(x='x', y='y', inner_radius=(n.level * width), outer_radius=((n.level * width) + width),
                             start_angle=n.right_bound, end_angle=(n.right_bound+n.range))
    n_old = n

In [ ]:
N = 9
x = np.linspace(-2, 2, N)
y = x**2
r = x/12.0+0.4

source = ColumnDataSource(dict(x=x, y=y, r=r))

xdr = DataRange1d()
ydr = DataRange1d()

plot = Plot(
    title=None, x_range=xdr, y_range=ydr, plot_width=300, plot_height=300,
    h_symmetry=False, v_symmetry=False, min_border=0, toolbar_location=None)

glyph = AnnularWedge(x="x", y="y", inner_radius=0.2, outer_radius="r", start_angle=0, end_angle=4, fill_color="#8888ee")
plot.add_glyph(source, glyph)

xaxis = LinearAxis()
plot.add_layout(xaxis, 'below')

yaxis = LinearAxis()
plot.add_layout(yaxis, 'left')

plot.add_layout(Grid(dimension=0, ticker=xaxis.ticker))
plot.add_layout(Grid(dimension=1, ticker=yaxis.ticker))

curdoc().add_root(plot)

show(plot)

In [ ]:
p = 0
p_old = 1
for i in range(10):
    p = p + p_old
    p_old = p
p, p_old

In [25]:
source = ColumnDataSource()

xdr = DataRange1d(start=5, end=15)
ydr = DataRange1d(start=5, end=15)

plot = Plot(title=None, x_range=xdr, y_range=ydr, plot_width=300, plot_height=300,
           h_symmetry=False, v_symmetry=False, min_border=0, toolbar_location=None)

Sunburst(a)

xaxis = LinearAxis()
plot.add_layout(xaxis, 'below')

yaxis = LinearAxis()
plot.add_layout(yaxis, 'left')

plot.add_layout(Grid(dimension=0, ticker=xaxis.ticker))
plot.add_layout(Grid(dimension=1, ticker=yaxis.ticker))

curdoc().add_root(plot)

show(plot)

In [23]:
n.__dict__

{'children': None,
 'leaves_subtree': 1,
 'left_bound': 6.283185307179586,
 'level': 3,
 'parent': <__main__.Node at 0x1e445ad3fd0>,
 'range': 0.8975979010256552,
 'right_bound': 5.385587406153931,
 'width': 1}

In [ ]:
a.children.append(c)

In [10]:
class Node:
    def __init__(self, level, children, leaves_subtree):
        self.level = level
        self.children = children
        self.leaves_subtree = leaves_subtree
        self.width = 1
        self.range = 1
        self.right_bound = 0
        self.left_bound = self.right_bound + self.range
        self.parent = None
    

In [11]:
def Set_parent(root):
    if root.children:
        for n in root.children:
            n.parent = root
            Set_parent(n)

In [12]:
def Sunburst(root):
    root.range = np.pi * 2
    root.right_bound = 0
    glyph = Ellipse(x=10, y=10, width=root.width * 2, height=root.width * 2)
    plot.add_glyph(source, glyph)
    DFS_visit(root)    

In [13]:
def DFS_visit(node):
    if node.children:
        count = 0
        n_old = None
        for n in node.children:
            if count == 0:
                n.right_bound = n.parent.right_bound
                for n in node.children:
                    n.range = (n.leaves_subtree/ n.parent.leaves_subtree) * n.parent.range
            else:
                n.right_bound = n_old.left_bound
            n.left_bound = n.right_bound + n.range
            glyph = AnnularWedge(x=10, y=10, inner_radius=(n.level * n.width), outer_radius=((n.level * n.width) + n.width),
                                 start_angle=n.right_bound, end_angle=(n.right_bound + n.range))
            plot.add_glyph(source, glyph)
            n_old = n
            count += 1
            DFS_visit(n)

In [14]:
m = Node(3, None, 1)
n = Node(3, None, 1)
l = Node(2, [m], 1)
f = Node(2, None, 1)
g = Node(2, None, 1)
h = Node(2, [n], 1)
i = Node(2, None, 1)
j = Node(2, None, 1)
k = Node(2, None, 1)
e = Node(1, [l,k,j], 3)
b = Node(1, [f], 1)
c = Node(1, [g, h], 2)
d = Node(1, [i], 1)
a = Node(0, [b,c,d,e], 7)

In [15]:
Set_parent(a)

In [ ]:
Sunburst(a)

In [ ]:
DFS_visit(a)

In [ ]:
derp = [1,1,1,1,1,1]
enumerate(derp)
t=0
for d in derp:
    if enumerate(derp) == 2:
        t = t*d
    else:
        t = t+d
        
t